In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import AutoImageProcessor, AutoModelForDepthEstimation
from peft import LoraConfig, get_peft_model

# ================= CONFIGURATION =================
FORCE_CPU = True # Mettre True pour test local
MODEL_ID = "depth-anything/Depth-Anything-V2-Small-hf"
OUTPUT_DIR = "./resultats_projet_v2"
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 10

if FORCE_CPU:
    DEVICE = "cpu"
else:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================= DATASET =================
class ZividDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.img_dir = os.path.join(root_dir, "images")
        self.depth_dir = os.path.join(root_dir, "depth")
        self.processor = processor
        valid_ext = ('.png', '.jpg', '.jpeg')
        if not os.path.exists(self.img_dir): raise FileNotFoundError(f"❌ Dossier introuvable : {self.img_dir}")
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.lower().endswith(valid_ext)])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        base_name = os.path.splitext(img_name)[0].replace("_color", "")
        npy_name = base_name + "_rawDepth.npy"
        img_path = os.path.join(self.img_dir, img_name)
        npy_path = os.path.join(self.depth_dir, npy_name)

        image = Image.open(img_path).convert("RGB")
        try:
            point_cloud = np.load(npy_path)
            depth_Z = point_cloud[:, :, 2]
            # Conversion mm -> mètres
            if np.nanmax(depth_Z) > 100: depth_Z = depth_Z / 1000.0
        except: return None

        inputs = self.processor(images=image, return_tensors="pt")
        target_h, target_w = inputs["pixel_values"].shape[-2:]
        depth_tensor = torch.from_numpy(depth_Z).float().unsqueeze(0).unsqueeze(0)
        mask = ~torch.isnan(depth_tensor) & ~torch.isinf(depth_tensor) & (depth_tensor > 0)
        depth_tensor = torch.nan_to_num(depth_tensor, nan=0.0)
        
        depth_resized = torch.nn.functional.interpolate(depth_tensor, size=(target_h, target_w), mode='nearest')
        mask_resized = torch.nn.functional.interpolate(mask.float(), size=(target_h, target_w), mode='nearest')
        
        return {"pixel_values": inputs["pixel_values"].squeeze(0), "labels": depth_resized.squeeze(0), "mask": mask_resized.squeeze(0)}

# ================= MÉTRIQUES =================
def compute_metrics(pred, target, mask):
    pred = pred[mask]
    target = target[mask]
    if len(target) == 0: return 0.0, 0.0, 0.0, 0.0

    # 1. AbsRel
    abs_rel = torch.mean(torch.abs(pred - target) / target)
    # 2. RMSE
    rmse = torch.sqrt(torch.mean((pred - target) ** 2))
    # 3. Log RMSE (Nouveau !)
    log_rmse = torch.sqrt(torch.mean((torch.log1p(pred) - torch.log1p(target)) ** 2))
    # 4. Delta < 1.25
    max_ratio = torch.max(pred / target, target / pred)
    delta1 = (max_ratio < 1.25).float().mean()

    return abs_rel.item(), rmse.item(), log_rmse.item(), delta1.item()

# ================= VISUALISATION =================
def save_curves(history):
    epochs = range(1, len(history['loss']) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Courbe Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['loss'], 'b-o', label='Training Loss')
    plt.title("Évolution de la Loss (MSE)")
    plt.xlabel("Époques"); plt.ylabel("Loss")
    plt.grid(True)
    
    # Courbe Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['delta'], 'g-o', label='Accuracy (Delta < 1.25)')
    plt.title("Évolution de la Précision")
    plt.xlabel("Époques"); plt.ylabel("Delta %")
    plt.grid(True)
    
    plt.savefig(os.path.join(OUTPUT_DIR, "courbes_entrainement.png"))
    plt.close()

def evaluate_full_dataset(model, loader, desc="Evaluation"):
    """Évalue le modèle sur tout le dataset sans entraînement"""
    model.eval()
    total_metrics = {'abs_rel': 0, 'rmse': 0, 'log_rmse': 0, 'delta': 0}
    count = 0
    
    print(f"📊 Lancement : {desc}...")
    with torch.no_grad():
        for batch in tqdm(loader, desc=desc):
            pixel_values = batch["pixel_values"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            mask = batch["mask"].to(DEVICE)
            
            outputs = model(pixel_values=pixel_values)
            prediction = torch.nn.functional.interpolate(
                outputs.predicted_depth.unsqueeze(1), size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            
            ar, rm, lrm, d1 = compute_metrics(prediction, labels, mask.bool())
            total_metrics['abs_rel'] += ar
            total_metrics['rmse'] += rm
            total_metrics['log_rmse'] += lrm
            total_metrics['delta'] += d1
            count += 1
            
    return {k: v/count for k, v in total_metrics.items()}

# ================= MAIN =================
def run_project():
    base_dir = os.getcwd()
    dataset_dir = os.path.join(base_dir, "DATASET_DEVOIR")
    
    processor = AutoImageProcessor.from_pretrained(MODEL_ID)
    model = AutoModelForDepthEstimation.from_pretrained(MODEL_ID)
    
    # --- 1. CONFIG LoRA ---
    lora_config = LoraConfig(r=16, lora_alpha=16, target_modules=["query", "value"], lora_dropout=0.1, bias="none")
    model = get_peft_model(model, lora_config)
    model.to(DEVICE)

    # --- 2. DATASET ---
    try:
        dataset = ZividDataset(dataset_dir, processor)
        dataset = [d for d in dataset if d is not None]
    except Exception as e: print(e); return
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # --- 3. EVALUATION AVANT ENTRAINEMENT (ZERO-SHOT) ---
    print("🛑 Évaluation du modèle AVANT Fine-Tuning (Zero-Shot)...")
    metrics_before = evaluate_full_dataset(model, loader, desc="Zero-Shot Eval")
    print(f"Scores initiaux : {metrics_before}")

    # --- 4. ENTRAINEMENT ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    history = {'loss': [], 'delta': []}
    
    model.train()
    print(f"🔥 Début du Fine-Tuning sur {EPOCHS} époques...")
    for epoch in range(1, EPOCHS + 1):
        epoch_loss = 0
        epoch_delta = 0
        loop = tqdm(loader, desc=f"Ep {epoch}/{EPOCHS}")
        
        for batch in loop:
            pixel_values = batch["pixel_values"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            mask = batch["mask"].to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(pixel_values=pixel_values)
            prediction = torch.nn.functional.interpolate(outputs.predicted_depth.unsqueeze(1), size=labels.shape[-2:], mode="bilinear", align_corners=False)
            
            loss = torch.sum(mask * (prediction - labels)**2) / (torch.sum(mask) + 1e-6)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                _, _, _, d1 = compute_metrics(prediction, labels, mask.bool())
            
            epoch_loss += loss.item()
            epoch_delta += d1
            loop.set_postfix(Loss=f"{loss.item():.3f}", Delta=f"{d1:.3f}")
        
        avg_loss = epoch_loss/len(loader)
        avg_delta = epoch_delta/len(loader)
        history['loss'].append(avg_loss)
        history['delta'].append(avg_delta)
        
    # --- 5. RESULTATS & SAUVEGARDE ---
    model.save_pretrained(os.path.join(OUTPUT_DIR, "modele_final_lora"))
    save_curves(history) # Génère l'image des courbes
    
    print("\n" + "="*30)
    print("📈 BILAN AVANT / APRÈS")
    print(f"AVANT (Zero-Shot) -> Delta: {metrics_before['delta']:.4f} | RMSE: {metrics_before['rmse']:.4f}")
    print(f"APRÈS (Fine-Tuned)-> Delta: {history['delta'][-1]:.4f} | RMSE: {np.sqrt(history['loss'][-1]):.4f}") # Approx RMSE from loss
    print("="*30)

if __name__ == "__main__":
    run_project()

🛑 Évaluation du modèle AVANT Fine-Tuning (Zero-Shot)...
📊 Lancement : Zero-Shot Eval...


Zero-Shot Eval: 100%|██████████| 15/15 [00:26<00:00,  1.75s/it]


Scores initiaux : {'abs_rel': 1.2894021034240724, 'rmse': 1.794843610127767, 'log_rmse': 0.5735661745071411, 'delta': 0.16639939645926158}
🔥 Début du Fine-Tuning sur 10 époques...


Ep 10/10: 100%|██████████| 15/15 [01:43<00:00,  6.91s/it, Delta=0.883, Loss=0.109]



📈 BILAN AVANT / APRÈS
AVANT (Zero-Shot) -> Delta: 0.1664 | RMSE: 1.7948
APRÈS (Fine-Tuned)-> Delta: 0.7526 | RMSE: 0.3187
